In [1]:
import openpyxl
import pandas as pd
import time

In [2]:
# the train information of users
loc = './data/four/tianchi_fresh_comp_train'
user_source_train1 = pd.read_excel(loc + '_user_train1.xlsx')
user_source_train2 = pd.read_excel(loc + '_user_train2.xlsx')
user_source_train3 = pd.read_excel(loc + '_user_train3.xlsx')
user_source_train4 = pd.read_excel(loc + '_user_train4.xlsx')
# the test information of users
user_source_test = pd.read_excel(loc + '_user_test.xlsx')
# the information of items
item_source = pd.read_excel('./data/four/tianchi_fresh_comp_train_item.xlsx')
# the user-item pair 
pair_file_test = pd.read_excel('./data/four/user_item_pair_test_form.xlsx')
pair_file_train = pd.read_excel('./data/four/user_item_pair_train_form.xlsx')

In [3]:
file_dict = {'18': user_source_test,
            '17': user_source_train1,
            '10': user_source_train2,
            '09': user_source_train3,
            '08': user_source_train4}

# Feature Generator

## User-Category Features

### Features of statistic

Numbers of items the user had interactions with in that category

In [4]:
def num_interactions(user_id, data_source, category_type):
    return [len(data_source.index)]

Repurchasing pattern: 1 if the user had repurchasing behavior

In [5]:
def repurchasing(user_id, data_source, category_type):
    df_4 = df_2[data_source['behavior_type'] == 4]
    if len(df_4.index) > 1:
        return [1]
    else:
        return [0]

Number of purchasing behaviors in that category

In [6]:
def num_purchasing(user_id, data_source, category_type):
    df_4 = data_source[data_source['behavior_type'] == 4]
    return [len(df_4.index)]

### Features of ratio

Category perference: Numbers of items the user had interactions with in that category divided by number of all items the user had interactions with

In [7]:
def cate_preference(user_id, data_source, category_type):
    return [len(data_source.index)/len(df_1.index)]

Category purchase power: Number of purchasing behaviors in that category divided by numbers of items the user had interactions with in that category

In [8]:
def cate_purchase_power(user_id, data_source, category_type):
    df_4 = data_source[data_source['behavior_type'] == 4]
    return [len(df_4.index)/len(data_source.index)]

Overnight pruchase pattern: whether the user purchase the item one day after browsing, collecting or adding into cart

In [9]:
def overnight(user_id, data_source, category_type):
    df_4 = data_source[data_source['behavior_type'] == 4]
    count = 0
    if len(df_4.index) == 0:
        return [0]
    else:
        for i in range(len(df_4.index)):
            overnight = int(df_4['time'][df_4.index[i]][8:10])-1
            for j in range(len(data_source.index)):
                if data_source['behavior_type'][data_source.index[j]] != 4 and int(data_source['time'][data_source.index[j]][8:10]) == overnight:
                    count += 1
        return [count/len(df_4.index)]
    

# Write in the pair file

In [10]:
feature_list = ['5_Number_of_items', '5_Repurchasing_pattern', '5_Number_of_purchasing', 
                '5_Category_prefernce', '5_Category_purchase_power', '5_Overnight_purchase_pattern']

In [11]:
def correcting(day, source):
    new_source = source
    del_list = []
    for i in range(len(new_source['user_id'])):
        if new_source['time'][i][9] == day:
            del_list.append(i)
    new_source = new_source.drop(del_list)
    return new_source

In [12]:
def all_feature(user_id, correcting_source, category_type):
    value_list = (num_interactions(user_id, correcting_source, category_type)+
                 repurchasing(user_id, correcting_source, category_type)+
                 num_purchasing(user_id, correcting_source, category_type)+
                 cate_preference(user_id, correcting_source, category_type)+
                 cate_purchase_power(user_id, correcting_source, category_type)+
                 overnight(user_id, correcting_source, category_type))
    return value_list

In [13]:
time_start=time.time()
wb_test = openpyxl.load_workbook(filename= './data/four/user_item_pair_test_form.xlsx')
sheet_test = wb_test['sheet1']
last_time, last_user, last_df = 0, 0, 0
for col_num in range(len(feature_list)):
            sheet_test.cell(row = 1, column = col_num +25, value=feature_list[col_num])

for row_num in range(len(pair_file_test.index)):
    day = pair_file_test['time'][row_num][8:10]
    user_id = pair_file_test['user_id'][row_num]
    if day == last_time and user_id == last_user:
        df_1 = last_df
    else:
        ori_source = file_dict[day]
        correcting_source = correcting(day, ori_source)
        df_1 = correcting_source[correcting_source['user_id']==user_id]
    
    item_id = pair_file_test['item_id'][row_num]
    df_2 = df_1[df_1['item_id']==item_id]
    category_type = df_2['item_category'][df_2.index[0]]
    df_3 = df_1[df_1['item_category'] == category_type]
    
    last_time = day
    last_user = user_id
    last_df = df_1
    
    value_list = all_feature(user_id, df_3, category_type)
    for col_num in range(len(feature_list)):
        sheet_test.cell(row = row_num+2, column = col_num +25, value=value_list[col_num])
        
        
wb_test.save('./data/four/user_item_pair_test_form.xlsx')
time_end=time.time()
print('all time',time_end-time_start)


E:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


all time 8502.766943216324


In [14]:
time_start=time.time()
wb_train = openpyxl.load_workbook(filename= './data/four/user_item_pair_train_form.xlsx')
sheet_train = wb_train['sheet1']
last_time, last_user, last_df = 0, 0, 0
for col_num in range(len(feature_list)):
            sheet_train.cell(row = 1, column = col_num +25, value=feature_list[col_num])

for row_num in range(len(pair_file_train.index)):
    day = pair_file_train['time'][row_num][8:10]
    user_id = pair_file_train['user_id'][row_num]
    if day == last_time and user_id == last_user:
        df_1 = last_df
    else:
        ori_source = file_dict[day]
        correcting_source = correcting(day, ori_source)
        df_1 = correcting_source[correcting_source['user_id']==user_id]
    
    item_id = pair_file_train['item_id'][row_num]
    df_2 = df_1[df_1['item_id']==item_id]
    category_type = df_2['item_category'][df_2.index[0]]
    df_3 = df_1[df_1['item_category'] == category_type]
    
    last_time = day
    last_user = user_id
    last_df = df_1
    
    value_list = all_feature(user_id, df_3, category_type)
    for col_num in range(len(feature_list)):
        sheet_train.cell(row = row_num+2, column = col_num +25, value=value_list[col_num])

    
wb_train.save('./data/four/user_item_pair_train_form.xlsx')
time_end=time.time()
print('all time',time_end-time_start)

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


all time 15752.987613916397
